**Part 1.1**: In this part you will implement variation of Dijkstra’s algorithm. It is a popular shortest path algorithm where the current known shortest path to each node is updated once new path is identified. This updating is called relaxing and in a graph with 𝑛 nodes it can occur at most 𝑛 − 1 times. In this part implement a function dijkstra (graph, source, k) which takes the graph and source as an input and where
each node can be relaxed on only k times where, 0 < 𝑘 < 𝑁 − 1. This function returns a distance and path dictionary which maps a node (which is an integer) to the distance and the path (sequence of nodes).

In [53]:
import random

In [1]:
# weighted digraph 
class DWG:

    def __init__(self):
        self.adj = {}
        self.weights = {}

    def are_connected(self, node1, node2):
        for neighbour in self.adj[node1]:
            if neighbour == node2:
                return True
        return False

    def adjacent_nodes(self, node):
        return self.adj[node]

    def add_node(self, node):
        self.adj[node] = []

    def add_edge(self, node1, node2, weight):
        if node2 not in self.adj[node1]:
            self.adj[node1].append(node2)
        self.weights[(node1, node2)] = weight

    def w(self, node1, node2):
        if self.are_connected(node1, node2):
            return self.weights[(node1, node2)]

    def num_nodes(self):
        return len(self.adj)

In [2]:
# min pq necessary for djisktra's 
class Item:
    def __init__(self, value, key):
        self.value = value
        self.key = key

class MinHeap:
    def __init__(self, elements):
        self.heap = elements
        self.positions = {element.value: i for i, element in enumerate(elements)}
        self.size = len(elements)
        self.build_heap()

    def parent(self, i):
        return (i - 1) // 2

    def left(self, i):
        return 2 * i + 1

    def right(self, i):
        return 2 * i + 2

    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        self.positions[self.heap[i].value], self.positions[self.heap[j].value] = i, j

    def min_heapify(self, i):
        l = self.left(i)
        r = self.right(i)
        smallest = i
        if l < self.size and self.heap[l].key < self.heap[i].key:
            smallest = l
        if r < self.size and self.heap[r].key < self.heap[smallest].key:
            smallest = r
        if smallest != i:
            self.swap(i, smallest)
            self.min_heapify(smallest)

    def build_heap(self):
        for i in range(self.size // 2, -1, -1):
            self.min_heapify(i)

    def extract_min(self):
        min_element = self.heap[0]
        self.size -= 1
        self.heap[0] = self.heap[self.size]
        self.positions[self.heap[0].value] = 0
        self.heap.pop()
        self.min_heapify(0)
        return min_element

    def decrease_key(self, value, new_key):
        i = self.positions[value]
        if new_key < self.heap[i].key:
            self.heap[i].key = new_key
            while i > 0 and self.heap[self.parent(i)].key > self.heap[i].key:
                self.swap(i, self.parent(i))
                i = self.parent(i)

    def insert(self, element):
        self.size += 1
        self.heap.append(element)
        self.positions[element.value] = self.size - 1
        self.decrease_key(element.value, element.key)

    def is_empty(self):
        return self.size == 0

In [41]:
def dijkstra(g, source, k):
    paths = {source: [source]} 
    dist = {} 
    nodes = list(g.adj.keys())
    relax_count = {} 
        
    Q = MinHeap([])
    
    for i in g.adj.keys(): 
        relax_count[i] = 0 
        
    for node in nodes:
        Q.insert(Item(node, float("inf")))
        dist[node] = float("inf")
        
    Q.decrease_key(source, 0)
    
    while not Q.is_empty(): 
        current_element = Q.extract_min() 
        current_node = current_element.value
        dist[current_node] = current_element.key 
        for neighbour in g.adj[current_node]:
            if dist[current_node] + g.w(current_node, neighbour) < dist[neighbour] and relax_count[neighbour] < k:
                Q.decrease_key(neighbour, dist[current_node] + g.w(current_node, neighbour))
                dist[neighbour] = dist[current_node] + g.w(current_node, neighbour)
                paths[neighbour] = paths.get(current_node, []) + [neighbour]
                relax_count[neighbour] += 1 
                
    return dist, relax_count, paths

In [42]:
g = DWG() 
g.add_node(0)
g.add_node(1)
g.add_node(2)
g.add_node(3) 
g.add_edge(0, 1, 4)
g.add_edge(0, 2, 3)
g.add_edge(1, 2, 1)
g.add_edge(1, 3, 2)
g.add_edge(2, 3, 5)

In [45]:
source_node = 0
k = 2 # Relax each nodes k = 2 times
distances, relax_count, paths = dijkstra(g, source_node, k)
print(distances)
print(relax_count)
print(paths) 

{0: 0, 1: 4, 2: 3, 3: 6}
{0: 0, 1: 1, 2: 1, 3: 2}
{0: [0], 1: [0, 1], 2: [0, 2], 3: [0, 1, 3]}


**Part 1.2**: Consider the same restriction as previous and implement a variation of Bellman Ford’s algorithm. This means implement a function bellman_ford(graph, source, k) which take the graph and source as an input and finds the path where each node can be relaxed only k times, where, 0 < 𝑘 < 𝑁 − 1. This function also returns a distance and path dictionary which maps a node (which is an integer) to the distance and the path (sequence of nodes).

In [46]:
def bf(g, source, k): 
    paths = {source: [source]} 
    dist = {} 
    nodes = list(g.adj.keys())
    relax_count = {} 
    
    for i in g.adj.keys(): 
        relax_count[i] = 0 
    
    for node in nodes: 
        dist[node] = float("inf")
        
    dist[source] = 0 
    
    for _ in range(g.num_nodes() - 1):
        for node in nodes: 
            for neighbour in g.adj[node]: 
                if relax_count[neighbour] < k and dist[neighbour] > dist[node] + g.w(node,neighbour):
                    dist[neighbour] = dist[node] + g.w(node,neighbour) 
                    paths[neighbour] = paths.get(node, []) + [neighbour]
                    relax_count[neighbour] += 1
                                        
    return dist, relax_count, paths

In [50]:
g1 = DWG() 
g1.add_node(0)
g1.add_node(1)
g1.add_node(2)
g1.add_node(3) 
g1.add_edge(0, 1, 4)
g1.add_edge(0, 2, 3)
g1.add_edge(1, 2, 1)
g1.add_edge(1, 3, 2)
g1.add_edge(2, 3, 5)

source_node = 0
k = 1  # Relax each nodes k = 1 times
distances, relax_count, paths = bf(g1, source_node, k)
print(distances)
print(paths) 
print(relax_count)

{0: 0, 1: 4, 2: 3, 3: 6}
{0: [0], 1: [0, 1], 2: [0, 2], 3: [0, 1, 3]}
{0: 0, 1: 1, 2: 1, 3: 1}


**Part 1.3**: Design an experiment to analyze the performance of functions written in Part 1.1 and 1.2. You should consider factors like graph size, graph density and value of k, that impact the algorithm performance in terms of its accuracy, time and space complexity.

The proposed experiment will examine how many relaxations per node (k values) are needed to acquire a shortest path for both algorithms for both a graph. By comparing the number of relaxations per node for Dijkstra's and Bellman-Ford's variations, we can assess which algorithm performs better in terms of efficiency. Lower average relaxations per node indicate better performance as it implies fewer iterations needed to find the shortest paths. 

First, we will find the shortest path of a large graph using a normal Dijkstra's algorithm with no relaxation limits. 

In [65]:
def dijkstra_nolimit(g, source):
    paths = {source: [source]} 
    dist = {} 
    nodes = list(g.adj.keys())
        
    Q = MinHeap([])
    
    for i in g.adj.keys(): 
        relax_count[i] = 0 
        
    for node in nodes:
        Q.insert(Item(node, float("inf")))
        dist[node] = float("inf")
        
    Q.decrease_key(source, 0)
    
    while not Q.is_empty(): 
        current_element = Q.extract_min() 
        current_node = current_element.value
        dist[current_node] = current_element.key 
        for neighbour in g.adj[current_node]:
            if dist[current_node] + g.w(current_node, neighbour) < dist[neighbour]:
                Q.decrease_key(neighbour, dist[current_node] + g.w(current_node, neighbour))
                dist[neighbour] = dist[current_node] + g.w(current_node, neighbour)
                paths[neighbour] = paths.get(current_node, []) + [neighbour]
                
    return dist, paths

In [66]:
def generate_graph(num_nodes, density): 
    graph = DWG() 
    
    for i in range(num_nodes): 
        graph.add_node(i) 
        
    for i in range(num_nodes): 
        for j in range(num_nodes): 
            if i != j and random.random() < density: 
                weight = random.randint(1,50)
                graph.add_edge(i, j, weight) 
    
    return graph 

num_nodes = 270 
density = 0.4 
g = generate_graph(num_nodes, density)

In [68]:
distances, paths = dijkstra_nolimit(g, 0) 
full_paths = paths 